In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [2]:
import pandas as pd
import numpy as np
import optuna 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split

import catboost as cb
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score

In [3]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub=pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv")

In [4]:
train.drop(columns=['id','cat10','cat5','cat7','cat8'],inplace=True)
test.drop(columns=['id','cat10','cat5','cat7','cat8'],inplace=True)

In [5]:
train.columns

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat6', 'cat9', 'cat11',
       'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cont0',
       'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8',
       'cont9', 'cont10', 'target'],
      dtype='object')

In [6]:
cat = [col for col in train.columns if 'cat' in col]
cont = [col for col in test.columns if 'cont' in col]

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 27 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    300000 non-null  object 
 1   cat1    300000 non-null  object 
 2   cat2    300000 non-null  object 
 3   cat3    300000 non-null  object 
 4   cat4    300000 non-null  object 
 5   cat6    300000 non-null  object 
 6   cat9    300000 non-null  object 
 7   cat11   300000 non-null  object 
 8   cat12   300000 non-null  object 
 9   cat13   300000 non-null  object 
 10  cat14   300000 non-null  object 
 11  cat15   300000 non-null  object 
 12  cat16   300000 non-null  object 
 13  cat17   300000 non-null  object 
 14  cat18   300000 non-null  object 
 15  cont0   300000 non-null  float64
 16  cont1   300000 non-null  float64
 17  cont2   300000 non-null  float64
 18  cont3   300000 non-null  float64
 19  cont4   300000 non-null  float64
 20  cont5   300000 non-null  float64
 21  cont6   30

In [8]:
oe=OrdinalEncoder()
train[cat]=oe.fit_transform(train[cat])
test[cat]=oe.transform(test[cat])

In [9]:
train[cat]=train[cat].astype('uint8')
test[cat]=test[cat].astype('uint8')

In [10]:
X=train.drop(columns=['target'])
y=train['target']

In [11]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
sc=StandardScaler()
X_train[cont]=sc.fit_transform(X_train[cont])
X_test[cont]=sc.transform(X_test[cont])
test[cont]=sc.transform(test[cont])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [13]:
def objective(trial):

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        'n_estimators' : trial.suggest_int('n_estimators' , 1 , 9999),
        'min_child_samples' : trial.suggest_int('min_child_samples' , 1 , 100),
        'eval_metric': 'AUC',
#         'verbose':1
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=100)

    preds = gbm.predict_proba(X_test)[:,1]
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(valid_y, pred_labels)
    accuracy = roc_auc_score(y_test,preds)
    return accuracy

study = optuna.create_study(direction="maximize", study_name='catboost')
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-03-26 15:03:50,283] A new study created in memory with name: catboost
[I 2021-03-26 15:08:06,850] Trial 0 finished with value: 0.8916209791780143 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.05748837852477402, 'depth': 6, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'n_estimators': 8623, 'min_child_samples': 50, 'subsample': 0.6686516035482349}. Best is trial 0 with value: 0.8916209791780143.
[I 2021-03-26 15:11:52,597] Trial 1 finished with value: 0.8822507228044503 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.048415255626609026, 'depth': 2, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 3260, 'min_child_samples': 25}. Best is trial 0 with value: 0.8916209791780143.
[I 2021-03-26 15:13:18,103] Trial 2 finished with value: 0.8764572472435734 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.0535013412297386, 'depth': 3, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'n_est

Number of finished trials: 4
Best trial:
  Value: 0.8916209791780143
  Params: 
    objective: Logloss
    colsample_bylevel: 0.05748837852477402
    depth: 6
    boosting_type: Plain
    bootstrap_type: Bernoulli
    n_estimators: 8623
    min_child_samples: 50
    subsample: 0.6686516035482349


In [14]:
# CatBoostClassifier()

In [15]:
Params={
    'objective': 'CrossEntropy',
    'colsample_bylevel': 0.08473689807822948,
    'depth': 4,
    'boosting_type': 'Ordered',
    'bootstrap_type': 'MVS',
    'n_estimators': 8482,
    'min_child_samples': 64,
}